In [1]:
# Initiate Python in Colab environment
!pip install PyDrive

     |████████████████████████████████| 993kB 6.3MB/s 
  Created wheel for PyDrive: filename=PyDrive-1.3.1-cp36-none-any.whl size=27437 sha256=61c245e8653aabb55f013bdb769f94df0de2f7c1b59e3b30af6cf184bde21c46
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
# Import essentials
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
# Enable Google drive authentication and authorization to read from the google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Link the training set, download and unzip it
download = drive.CreateFile({'id': '1gvdIbq3G4JwHvRUGvW7KnWA9hP2bpUSy'})
download.GetContentFile('Full_Set_Proc_Comb.zip')
!unzip Full_Set_Proc_Comb.zip

In [0]:
# Link the training set
download = drive.CreateFile({'id': '1mM5e0SRKuTklmVoKtqKx8dT_mxezrhxo'})
download.GetContentFile('Test_Set_Proc_Comb.zip')
!unzip Test_Set_Proc_Comb.zip


In [0]:
# import keras libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [0]:
# Read the train and test files
train = pd.read_csv('Full_Set_Proc_Comb/train.csv')
test = pd.read_csv('Test_Set_Proc_Comb/test.csv')

In [13]:
# Get full file names
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
df = train
df['id'] = df.id.astype(str)
df['label'] = df.label.astype(str)
df['id'] = df['id'] + '.jpg'
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
train_set.head()

,id,label
382,383.jpg,1
538,539.jpg,1
1493,1494.jpg,4
1112,1113.jpg,3
324,325.jpg,1


In [14]:
# Get full file names for test set
df_test = test
df_test['id'] = df_test.id.astype(str)
df_test['label'] = df_test.label.astype(str)
df_test['id'] = df_test['id'] + '.jpg'
df_test.head()

,id,label
0,1501.jpg,0
1,1502.jpg,0
2,1503.jpg,0
3,1504.jpg,0
4,1505.jpg,0


In [15]:
# Check if data is split sort of evenly
train_set['label'].value_counts()

2    251
0    244
4    236
1    235
3    234
Name: label, dtype: int64

In [16]:
# Define Data Generators
datagen=ImageDataGenerator(
    rotation_range = 45,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    rescale=1./255)

datagentest=ImageDataGenerator(
    rescale=1./255)

train_generator=datagen.flow_from_dataframe(
    dataframe=train_set, 
    directory="Full_Set_Proc_Comb", 
    x_col="id", 
    y_col="label", 
    class_mode="categorical", 
    target_size=(100,100), 
    batch_size=32,
    shuffle=True)

valid_generator=datagentest.flow_from_dataframe(
    dataframe=test_set, 
    directory="Full_Set_Proc_Comb", 
    x_col="id", 
    y_col="label", 
    class_mode="categorical", 
    target_size=(100,100), 
    batch_size=32,
    shuffle=False)

test_generator=datagentest.flow_from_dataframe(
    dataframe=df_test, 
    directory="Test_Set_Proc_Comb", 
    x_col="id", 
    y_col="label", 
    class_mode="categorical", 
    target_size=(100,100), 
    batch_size=32,
    shuffle=False)

Found 1200 validated image filenames belonging to 5 classes.
Found 300 validated image filenames belonging to 5 classes.
Found 50 validated image filenames belonging to 5 classes.


In [17]:
# Define best model with Batch Normalization

from keras import layers
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),input_shape=(100,100,3), padding='same', use_bias = 'False'))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(Conv2D(32, (3, 3), use_bias = 'False'))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), use_bias = 'False', padding='same'))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(Conv2D(64, (3, 3), use_bias = 'False'))
model.add(layers.BatchNormalization(axis=3))
model.add(layers.Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, use_bias = 'False'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, use_bias = 'False'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('softmax'))








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Create a checkpoint for saving the optimum validation loss parameter
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('weights_cap2_3.hdf5', monitor='val_loss', save_best_only=True)
callbacks_list = [checkpoint]

In [19]:
# Compile the model
from keras import optimizers
model.compile(optimizers.rmsprop(lr=0.001),loss='categorical_crossentropy', metrics=["accuracy"])

In [20]:
%%time
### TRAIN THE MODEL
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=250,
                    callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/250
37/37 [==============================] - 9s 251ms/step - loss: 0.9363 - acc: 0.6613 - val_loss: 1.1243 - val_acc: 0.5868
Epoch 2/250
37/37 [==============================] - 5s 133ms/step - loss: 0.8153 - acc: 0.7179 - val_loss: 0.6633 - val_acc: 0.8022
Epoch 3/250
37/37 [==============================] - 5s 132ms/step - loss: 0.7515 - acc: 0.7432 - val_loss: 0.7506 - val_acc: 0.7463
Epoch 4/250
37/37 [==============================] - 5s 129ms/step - loss: 0.7227 - acc: 0.7618 - val_loss: 1.0910 - val_acc: 0.6269
Epoch 5/250
37/37 [==============================] - 5s 129ms/step - loss: 0.6613 - acc: 0.8159 - val_loss: 0.9445 - val_acc: 0.6530
Epoch 6/250
37/37 [==============================] - 5s 129ms/step - loss: 0.6676 - acc: 0.7922 - val_loss: 0.6887 - val_acc: 0.7425
Epoch 7/250
37/37 [==============================] - 5s 129ms/step - loss: 0.6009 - acc: 0.8302 - val_loss: 

In [21]:
# Evaluate the model on validation and test set
print(model.evaluate_generator(valid_generator))
model.evaluate_generator(test_generator)

[0.553207596540451, 0.8933333333333333]


[0.7143347668647766, 0.8400000023841858]

In [22]:
# Load the checkpoint in to a new model to compare
new_model = keras.models.load_model('weights_cap2_3.hdf5')
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 98, 98, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)       

In [23]:
# Evaluate the new model on validation and test set
print(new_model.evaluate_generator(valid_generator))
print(new_model.evaluate_generator(test_generator))

[0.14419285039727886, 0.9533333333333334]
[0.15934765964746475, 0.94]


In [26]:
# Get the confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
test_generator.reset()
Y_pred = new_model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

Confusion Matrix
[[ 8  0  1  0  1]
 [ 0 10  0  0  0]
 [ 0  1  9  0  0]
 [ 0  0  0 10  0]
 [ 0  0  0  0 10]]


In [27]:
# Get classification report
print('Classification Report')
target_names = ['Apple', 'Blackberry', 'Green Grapes', 'Kiwi', 'Strawberry']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

       Apple       1.00      0.80      0.89        10
  Blackberry       0.91      1.00      0.95        10
Green Grapes       0.90      0.90      0.90        10
        Kiwi       1.00      1.00      1.00        10
  Strawberry       0.91      1.00      0.95        10

    accuracy                           0.94        50
   macro avg       0.94      0.94      0.94        50
weighted avg       0.94      0.94      0.94        50

